In [1]:
from twelvedata import TDClient
from twelvedata.exceptions import (TwelveDataError, BadRequestError)
import time
import sys
import calendar

ticker = 'TSLA'
interval = '1min'
output_size = 5000
year = 2020
filename = f'{ticker}_{year}_{interval}.csv'

In [2]:
# Load Twelve Data API key
api_key = None

with open('twelve_data_api_key', 'r') as file:
    api_key = file.read()
    
if api_key is None:
    sys.exit("Error, no API key")

In [3]:
def query_td_api(instance, api_name, **kwargs):
    success = False
    running = True
    result = instance
    fail_count = 0
    while running:
        try:
            api = getattr(instance, api_name)
            result = api(**kwargs)
            running = False
            success = True
            
        except BadRequestError as e:
            # In case of a bad request, we do not retry
            print(e, ".. ", end='')
            running = False
            
        except TwelveDataError as e:
            
            if fail_count == 0:
                print("still working ..", end='')

                # Only wait until the next minute begins (+1 second for good measure)
                time.sleep(60 - time.localtime().tm_sec + 1)
                fail_count = fail_count + 1
            
            else:
                # We've failed twice, we should not retry
                running = False
                print("API {} failed..".format(api_name), end='')
                
    return (result, success)

In [4]:
# Let's fill this in with a month-by-month dateranges to feed to the API
# This is broken up just so we don't fill up the 5000 outputsize maximum APIlimit.
date_ranges = []

# 12 entries starting from 1 for the month
# calendar.monthrange gives us (start-date, end-date) for the month in question
for month in range(1, 13):
    daterange = calendar.monthrange(year, month)
    date_ranges.append((f'{year}-{month}-01', f'{year}-{month}-{daterange[1]}'))

# Because the output is ordered descending in date, let's reverse this list so we get december first.
date_ranges.reverse()

td = TDClient(apikey=api_key)
csv_data = []
csv_headers = None
iteration_count = 0

for date_range in date_ranges:
    print(f'Fetching data for date range: {date_range}.. ', end='')
    ts = td.time_series(symbol=ticker,
                        interval=interval,
                        start_date=date_range[0],
                        end_date=date_range[1],
                        outputsize=output_size)

    # Technical Indicators
#     api_calls = ['with_adx', 'with_bbands', 'with_ema', 'with_ichimoku', 'as_csv']
    api_calls = ['with_adx', 'with_ema', 'as_csv']
    status = True
    for api in api_calls:
        (ts, status) = query_td_api(ts, api)
        if status is False:
            break
    if status is True:
        if csv_headers is None:
            csv_headers = ts[0]

        csv_data.extend(ts[1:])

    print('done')

csv_data.insert(0, csv_headers)

Fetching data for date range: ('2020-12-01', '2020-12-31').. done
Fetching data for date range: ('2020-11-01', '2020-11-30').. done
Fetching data for date range: ('2020-10-01', '2020-10-31').. still working ..done
Fetching data for date range: ('2020-9-01', '2020-9-30').. done
Fetching data for date range: ('2020-8-01', '2020-8-31').. still working ..done
Fetching data for date range: ('2020-7-01', '2020-7-31').. done
Fetching data for date range: ('2020-6-01', '2020-6-30').. still working ..done
Fetching data for date range: ('2020-5-01', '2020-5-31').. done
Fetching data for date range: ('2020-4-01', '2020-4-30').. still working ..done
Fetching data for date range: ('2020-3-01', '2020-3-31').. done
Fetching data for date range: ('2020-2-01', '2020-2-29').. No data is available on the specified dates. Try setting different start/end dates. .. done
Fetching data for date range: ('2020-1-01', '2020-1-31').. No data is available on the specified dates. Try setting different start/end dat

In [5]:
csv_data.insert(0, csv_headers)
# Write data to CSV file.
import csv
with open(filename, 'w') as file:
    writer = csv.writer(file)
    writer.writerows(csv_data)
file.close()